In [1]:
import os
import util
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Input, TimeDistributed, GRU, Dense, Dropout, Lambda, GlobalAveragePooling2D, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score

In [2]:
base_dir = 'artifacts'
data = os.path.join(base_dir, 'split_3d_data.pkl')
batch_size = 32

In [3]:
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Load Data

In [4]:
X_train, X_val, X_test, y_train, y_val, y_test = util.load_split_3d_data(data)

In [5]:
X_train.shape, X_test.shape, X_val.shape

((280, 16, 224, 224, 3), (60, 16, 224, 224, 3), (60, 16, 224, 224, 3))

In [6]:
num_frames, img_size = X_train.shape[1], X_train.shape[2:4]
print(num_frames, img_size)

16 (224, 224)


In [7]:
y_train.shape, y_test.shape, y_val.shape

((280,), (60,), (60,))

In [8]:
num_train, num_val, num_test = X_train.shape[0], X_val.shape[0], X_test.shape[0]
num_train, num_val, num_test

(280, 60, 60)

## Factorizing Target

In [9]:
labels = pd.factorize(y_val)[1]
print(labels)

['real' 'fake']


In [10]:
y_train, y_val, y_test = pd.factorize(y_train)[0], pd.factorize(y_val)[0], pd.factorize(y_test)[0]

In [11]:
y_train.shape, y_test.shape, y_val.shape

((280,), (60,), (60,))

# MobileNetV2 & GRU

In [12]:
base_cnn = keras.applications.MobileNetV2(
    input_shape=img_size+(3,),
    include_top=False,
    weights='imagenet')
# unfreezing a few of the last layers
base_cnn.trainable = True
for layer in base_cnn.layers[:-30]:
    layer.trainable = False

In [13]:
# adding data augmentation for make model more robust
data_augmentation = Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1),
    keras.layers.RandomBrightness(0.2),
    keras.layers.RandomContrast(0.2)
])

In [14]:
(num_frames,)+img_size+(3,)

(16, 224, 224, 3)

In [15]:
model = Sequential()
model.add(Input(shape=(num_frames,)+img_size+(3,)))
# Using TimeDistributed to apply Augmentation, MobileNetV2 Preprocessing, & MobileNetV2 CNN frame-by-frame
model.add(TimeDistributed(data_augmentation))
model.add(TimeDistributed(Lambda(keras.applications.mobilenet_v2.preprocess_input)))
model.add(TimeDistributed(base_cnn))
model.add(TimeDistributed(GlobalAveragePooling2D()))
model.add(GRU(256, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed (TimeDistributed)   │ (None, 16, 224, 224, 3)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 16, 224, 224, 3)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ (None, 16, 7, 7, 1280)      │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_3 (TimeDistributed) │ (None, 16, 1280)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 256)                 │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,472,193 (13.25 MB)

 Trainable params: 2,740,609 (10.45 MB)

 Non-trainable params: 731,584 (2.79 MB)

In [16]:
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [15]:
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=5,
                      restore_best_weights=True, verbose=1)
model.fit(X_train, y_train, 
           validation_data=(X_val, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 177s 10s/step - accuracy: 0.4536 - loss: 0.9709 - val_accuracy: 0.5333 - val_loss: 0.7297
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 61s 7s/step - accuracy: 0.5321 - loss: 0.8538 - val_accuracy: 0.5167 - val_loss: 0.7160
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 61s 7s/step - accuracy: 0.5214 - loss: 0.8434 - val_accuracy: 0.5500 - val_loss: 0.7121
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 62s 7s/step - accuracy: 0.4821 - loss: 0.8644 - val_accuracy: 0.5000 - val_loss: 0.7065
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 61s 7s/step - accuracy: 0.5357 - loss: 0.8430 - val_accuracy: 0.4667 - val_loss: 0.7049
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 60s 7s/step - accuracy: 0.5143 - loss: 0.8350 - val_accuracy: 0.4667 - val_loss: 0.7067
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 61s 7s/step - accuracy: 0.5107 - loss: 0.8367 - val_accuracy: 0.4500 - val_loss: 0.7050
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 61s 7s/step - accuracy: 0.5643 - loss: 0.7892 - val_accuracy: 0.4333 - val_loss: 0.703

In [16]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy:", test_accuracy)
print("Test Loss:", test_loss)

Test Accuracy: 0.4833333194255829
Test Loss: 0.7482662796974182


In [20]:
model.save('artifacts/mobilenetv2_gru.keras')

**Didn't perform as good as the majority voting technique. The reason is the small size of the dataset (just 400 videos in total). Deep Neural Networks particularly one that includes some type of RNNs will require several hundred thousands of samples to train and generalize well.**

* We know that the MobileNetV2 model is able to extract features from the frames, because it is already trained on ImageNet (1.2M images), hence, it knows how to see.
* So, instead of training that base model again and again and wasting the 400 videos teaching it basic vision, we can simply obtain those features as 'embeddings'.Then these embeddings can be passed to the GRU or any other model for the final classification task.
* The use of this is that we can add regularization and try tuning without having to include the training of base-CNN as well, reducing the training time and number of learnable parameters.
* Each embedding vector will be like a semantic summary of the video frames. So, feeding those instead of raw pixels, should make it easier for the classifier to learn, even with little data.

# Embeddings & GRU

In [12]:
def get_embeddings(X_frames, batch_size, num_data, num_frames):
    base_cnn = keras.applications.MobileNetV2(
        input_shape=img_size+(3,),
        include_top=False,
        weights='imagenet',
        pooling='avg')
    preprocessed = keras.applications.mobilenet_v2.preprocess_input(X_frames)
    embeddings = base_cnn.predict(preprocessed, batch_size=batch_size, verbose=1)
    print("Embeddings shape:",embeddings.shape)
    return embeddings.reshape(num_data, num_frames, -1)

In [13]:
train_frames, val_frames, test_frames = util.convert_3d_to_2d(split=3,
                                                         train=(X_train, y_train),
                                                         val=(X_val,y_val),
                                                         test=(X_test, y_test))
X_train_frames, _ = train_frames
X_val_frames, _ = val_frames
X_test_frames, _ = test_frames

In [14]:
train_embeddings = get_embeddings(X_train_frames, batch_size, num_train, num_frames)
train_embeddings.shape

140/140 ━━━━━━━━━━━━━━━━━━━━ 34s 234ms/step
Embeddings shape: (4480, 1280)


(280, 16, 1280)

In [15]:
val_embeddings = get_embeddings(X_val_frames, 8, num_val, num_frames)
val_embeddings.shape

120/120 ━━━━━━━━━━━━━━━━━━━━ 9s 65ms/step
Embeddings shape: (960, 1280)


(60, 16, 1280)

In [16]:
test_embeddings = get_embeddings(X_val_frames, 8, num_val, num_frames)
test_embeddings.shape

120/120 ━━━━━━━━━━━━━━━━━━━━ 9s 67ms/step
Embeddings shape: (960, 1280)


(60, 16, 1280)

In [17]:
num_features = train_embeddings.shape[-1]
num_features

1280

In [18]:
model1 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(128, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
model1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 128)                 │         541,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 541,569 (2.07 MB)

 Trainable params: 541,569 (2.07 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model1.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model1.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.5000 - loss: 0.8411 - val_accuracy: 0.5500 - val_loss: 0.7246
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4929 - loss: 0.8049 - val_accuracy: 0.5667 - val_loss: 0.7208
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.4750 - loss: 0.8221 - val_accuracy: 0.5500 - val_loss: 0.7183
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5036 - loss: 0.7696 - val_accuracy: 0.5667 - val_loss: 0.7161
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4857 - loss: 0.7918 - val_accuracy: 0.5500 - val_loss: 0.7142
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.4750 - loss: 0.8146 - val_accuracy: 0.5500 - val_loss: 0.7125
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.4571 - loss: 0.8056 - val_accuracy: 0.5500 - val_loss: 0.7106
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5143 - loss: 0.7901 - val_accuracy: 0.5500 - val_loss:

In [20]:
test_loss1, test_accuracy1 = model1.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy1)
print("Test Loss:", test_loss1)

Test Accuracy: 0.550000011920929
Test Loss: 0.678394615650177


In [21]:
model2 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_1 (GRU)                          │ (None, 256)                 │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,214,209 (4.63 MB)

 Trainable params: 1,214,209 (4.63 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model2.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model2.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.5821 - loss: 0.6854 - val_accuracy: 0.6000 - val_loss: 0.6745
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5500 - loss: 0.7121 - val_accuracy: 0.6000 - val_loss: 0.6747
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5893 - loss: 0.6760 - val_accuracy: 0.6000 - val_loss: 0.6745
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5536 - loss: 0.7203 - val_accuracy: 0.6000 - val_loss: 0.6748
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5714 - loss: 0.6739 - val_accuracy: 0.6000 - val_loss: 0.6746
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5964 - loss: 0.6815 - val_accuracy: 0.6167 - val_loss: 0.6744
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5464 - loss: 0.7117 - val_accuracy: 0.6000 - val_loss: 0.6746
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.6250 - loss: 0.6703 - val_accuracy: 0.6000 - val_loss:

In [24]:
test_loss2, test_accuracy2 = model2.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy2)
print("Test Loss:", test_loss2)

Test Accuracy: 0.6166666746139526
Test Loss: 0.6736595630645752


In [25]:
model3 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model3.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_2 (GRU)                          │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,576,193 (6.01 MB)

 Trainable params: 1,576,193 (6.01 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
model3.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model3.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 97ms/step - accuracy: 0.4679 - loss: 0.7421 - val_accuracy: 0.5000 - val_loss: 0.6825
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.4750 - loss: 0.7371 - val_accuracy: 0.5167 - val_loss: 0.6816
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5250 - loss: 0.7201 - val_accuracy: 0.5333 - val_loss: 0.6808
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5500 - loss: 0.7013 - val_accuracy: 0.5167 - val_loss: 0.6805
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.5357 - loss: 0.7068 - val_accuracy: 0.5167 - val_loss: 0.6804
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5786 - loss: 0.7058 - val_accuracy: 0.5000 - val_loss: 0.6803
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5357 - loss: 0.6738 - val_accuracy: 0.5333 - val_loss: 0.6791
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5071 - loss: 0.7150 - val_accuracy: 0.5333 - val_loss:

In [27]:
test_loss3, test_accuracy3 = model3.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy3)
print("Test Loss:", test_loss3)

Test Accuracy: 0.5333333611488342
Test Loss: 0.6725301146507263


In [28]:
model4 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model4.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_4 (GRU)                          │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_5 (GRU)                          │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,608,961 (6.14 MB)

 Trainable params: 1,608,961 (6.14 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
model4.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model4.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 111ms/step - accuracy: 0.4714 - loss: 0.8271 - val_accuracy: 0.5500 - val_loss: 0.7211
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5143 - loss: 0.7776 - val_accuracy: 0.5000 - val_loss: 0.7118
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.4429 - loss: 0.7824 - val_accuracy: 0.4667 - val_loss: 0.7079
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.4679 - loss: 0.7790 - val_accuracy: 0.4500 - val_loss: 0.7064
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.4786 - loss: 0.7494 - val_accuracy: 0.4833 - val_loss: 0.7055
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5357 - loss: 0.7215 - val_accuracy: 0.4833 - val_loss: 0.7056
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5000 - loss: 0.7386 - val_accuracy: 0.5000 - val_loss: 0.7057
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.4643 - loss: 0.7756 - val_accuracy: 0.4833 - val_loss

In [30]:
test_loss4, test_accuracy4 = model4.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy4)
print("Test Loss:", test_loss4)

Test Accuracy: 0.46666666865348816
Test Loss: 0.7025130987167358


In [31]:
model5 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model5.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_6 (GRU)                          │ (None, 256)                 │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,001 (4.88 MB)

 Trainable params: 1,280,001 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
model5.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model5.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 0.4964 - loss: 0.8413 - val_accuracy: 0.4167 - val_loss: 0.7131
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4964 - loss: 0.8164 - val_accuracy: 0.3833 - val_loss: 0.7128
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.4821 - loss: 0.8542 - val_accuracy: 0.4333 - val_loss: 0.7128
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5357 - loss: 0.8168 - val_accuracy: 0.4667 - val_loss: 0.7128
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4893 - loss: 0.8714 - val_accuracy: 0.5000 - val_loss: 0.7129
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4607 - loss: 0.8340 - val_accuracy: 0.4833 - val_loss: 0.7124
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4750 - loss: 0.8643 - val_accuracy: 0.4667 - val_loss: 0.7123
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5143 - loss: 0.8430 - val_accuracy: 0.4833 - val_loss:

In [33]:
test_loss5, test_accuracy5 = model5.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy5)
print("Test Loss:", test_loss5)

Test Accuracy: 0.5833333134651184
Test Loss: 0.6836086511611938


In [34]:
model6 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=False),
    BatchNormalization(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='sigmoid'),
])
model6.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_7 (GRU)                          │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_8 (GRU)                          │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,611,521 (6.15 MB)

 Trainable params: 1,610,241 (6.14 MB)

 Non-trainable params: 1,280 (5.00 KB)

In [35]:
model6.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model6.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.5000 - loss: 0.8489 - val_accuracy: 0.4833 - val_loss: 0.7046
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.5571 - loss: 0.7599 - val_accuracy: 0.5000 - val_loss: 0.6995
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.6000 - loss: 0.6994 - val_accuracy: 0.4667 - val_loss: 0.6953
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.6393 - loss: 0.6492 - val_accuracy: 0.4167 - val_loss: 0.6915
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.6571 - loss: 0.6059 - val_accuracy: 0.4667 - val_loss: 0.6884
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.7036 - loss: 0.5680 - val_accuracy: 0.4833 - val_loss: 0.6859
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.7179 - loss: 0.5348 - val_accuracy: 0.5167 - val_loss: 0.6837
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.7464 - loss: 0.5056 - val_accuracy: 0.5667 - val_loss

In [36]:
test_loss6, test_accuracy6 = model6.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy6)
print("Test Loss:", test_loss6)

Test Accuracy: 0.6166666746139526
Test Loss: 0.6815556883811951


In [43]:
model7 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model7.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_15 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_16 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,674,753 (6.39 MB)

 Trainable params: 1,674,753 (6.39 MB)

 Non-trainable params: 0 (0.00 B)

In [44]:
model7.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model7.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 115ms/step - accuracy: 0.5214 - loss: 0.8035 - val_accuracy: 0.5000 - val_loss: 0.7401
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.4893 - loss: 0.7803 - val_accuracy: 0.5000 - val_loss: 0.7288
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.4750 - loss: 0.7688 - val_accuracy: 0.5000 - val_loss: 0.7211
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.4929 - loss: 0.7410 - val_accuracy: 0.5000 - val_loss: 0.7160
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.5214 - loss: 0.7421 - val_accuracy: 0.5000 - val_loss: 0.7125
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5179 - loss: 0.7330 - val_accuracy: 0.4833 - val_loss: 0.7106
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.4607 - loss: 0.7963 - val_accuracy: 0.4667 - val_loss: 0.7090
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.4750 - loss: 0.7637 - val_accuracy: 0.4000 - val_loss

In [45]:
test_loss7, test_accuracy7 = model7.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy7)
print("Test Loss:", test_loss7)

Test Accuracy: 0.6666666865348816
Test Loss: 0.673348605632782


In [76]:
model8 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model8.summary()

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_44 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_45 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_51 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_53 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_52 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_54 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_53 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_55 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_54 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_56 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,871,873 (7.14 MB)

 Trainable params: 1,871,873 (7.14 MB)

 Non-trainable params: 0 (0.00 B)

In [77]:
model8.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model8.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 108ms/step - accuracy: 0.5179 - loss: 0.7287 - val_accuracy: 0.5000 - val_loss: 0.6917
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.4929 - loss: 0.7360 - val_accuracy: 0.5000 - val_loss: 0.6915
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.5571 - loss: 0.7178 - val_accuracy: 0.5167 - val_loss: 0.6911
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.4786 - loss: 0.7715 - val_accuracy: 0.5333 - val_loss: 0.6912
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.4607 - loss: 0.7648 - val_accuracy: 0.5333 - val_loss: 0.6909
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.5214 - loss: 0.7225 - val_accuracy: 0.5167 - val_loss: 0.6907
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.5143 - loss: 0.7535 - val_accuracy: 0.5167 - val_loss: 0.6902
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.5036 - loss: 0.7419 - val_accuracy: 0.5167 - val_loss

In [78]:
test_loss8, test_accuracy8 = model8.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy8)
print("Test Loss:", test_loss8)

Test Accuracy: 0.5166666507720947
Test Loss: 0.6899269223213196


In [84]:
model9 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model9.summary()

Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_52 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_53 (GRU)                         │ (None, 16, 256)             │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_54 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_60 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_62 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_61 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_63 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,003,713 (7.64 MB)

 Trainable params: 2,003,713 (7.64 MB)

 Non-trainable params: 0 (0.00 B)

In [85]:
model9.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model9.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 138ms/step - accuracy: 0.5357 - loss: 0.7183 - val_accuracy: 0.4167 - val_loss: 0.6982
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.5143 - loss: 0.7126 - val_accuracy: 0.4667 - val_loss: 0.6959
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.4536 - loss: 0.7643 - val_accuracy: 0.4500 - val_loss: 0.6947
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.5143 - loss: 0.7260 - val_accuracy: 0.4333 - val_loss: 0.6940
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.5286 - loss: 0.6978 - val_accuracy: 0.4667 - val_loss: 0.6935
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.5393 - loss: 0.7159 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.4679 - loss: 0.7356 - val_accuracy: 0.5333 - val_loss: 0.6930
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.5036 - loss: 0.7262 - val_accuracy: 0.5500 - val_loss

In [86]:
test_loss9, test_accuracy9 = model9.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy9)
print("Test Loss:", test_loss9)

Test Accuracy: 0.5833333134651184
Test Loss: 0.6773016452789307


In [87]:
model10 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model10.summary()

Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_55 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_56 (GRU)                         │ (None, 16, 256)             │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_57 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_62 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_64 (Dense)                     │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_63 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_65 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_64 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_66 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,069,505 (7.89 MB)

 Trainable params: 2,069,505 (7.89 MB)

 Non-trainable params: 0 (0.00 B)

In [88]:
model10.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model10.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 150ms/step - accuracy: 0.4464 - loss: 0.7614 - val_accuracy: 0.5667 - val_loss: 0.6886
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.5714 - loss: 0.6875 - val_accuracy: 0.5333 - val_loss: 0.6886
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.5536 - loss: 0.6977 - val_accuracy: 0.5333 - val_loss: 0.6884
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.5714 - loss: 0.6819 - val_accuracy: 0.5000 - val_loss: 0.6880
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.5643 - loss: 0.7018 - val_accuracy: 0.5167 - val_loss: 0.6877
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.5250 - loss: 0.7064 - val_accuracy: 0.5000 - val_loss: 0.6872
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.4679 - loss: 0.7189 - val_accuracy: 0.5000 - val_loss: 0.6866
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.5286 - loss: 0.7186 - val_accuracy: 0.5000 - val_loss

In [89]:
test_loss10, test_accuracy10 = model10.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy10)
print("Test Loss:", test_loss10)

Test Accuracy: 0.6166666746139526
Test Loss: 0.6532056331634521


In [90]:
model11 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model11.summary()

Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_58 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_59 (GRU)                         │ (None, 16, 256)             │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_60 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_65 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_67 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_66 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_68 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_67 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_69 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_68 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_70 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,266,625 (8.65 MB)

 Trainable params: 2,266,625 (8.65 MB)

 Non-trainable params: 0 (0.00 B)

In [91]:
model11.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model11.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 152ms/step - accuracy: 0.4964 - loss: 0.7272 - val_accuracy: 0.4667 - val_loss: 0.6925
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.4893 - loss: 0.7428 - val_accuracy: 0.5000 - val_loss: 0.6925
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.4714 - loss: 0.7377 - val_accuracy: 0.4833 - val_loss: 0.6925
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.4857 - loss: 0.7285 - val_accuracy: 0.5000 - val_loss: 0.6922
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.5214 - loss: 0.7006 - val_accuracy: 0.5167 - val_loss: 0.6922
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.4750 - loss: 0.7267 - val_accuracy: 0.5000 - val_loss: 0.6921
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.4714 - loss: 0.7123 - val_accuracy: 0.4833 - val_loss: 0.6920
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.5107 - loss: 0.7187 - val_accuracy: 0.4833 - val_loss

In [92]:
test_loss11, test_accuracy11 = model11.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy11)
print("Test Loss:", test_loss11)

Test Accuracy: 0.5
Test Loss: 0.6916480660438538


In [93]:
model12 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model12.summary()

Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_61 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_62 (GRU)                         │ (None, 16, 256)             │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_63 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_69 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_71 (Dense)                     │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_70 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_72 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_71 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_73 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,071,553 (7.90 MB)

 Trainable params: 2,070,529 (7.90 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [94]:
model12.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model12.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 165ms/step - accuracy: 0.5321 - loss: 0.8883 - val_accuracy: 0.4667 - val_loss: 0.6977
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.5107 - loss: 0.8940 - val_accuracy: 0.4833 - val_loss: 0.6960
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.5036 - loss: 0.9210 - val_accuracy: 0.5167 - val_loss: 0.6944
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.5536 - loss: 0.8372 - val_accuracy: 0.5500 - val_loss: 0.6929
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.4786 - loss: 0.9163 - val_accuracy: 0.5500 - val_loss: 0.6916
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.5464 - loss: 0.8103 - val_accuracy: 0.5333 - val_loss: 0.6906
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.5536 - loss: 0.8293 - val_accuracy: 0.5167 - val_loss: 0.6896
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.5071 - loss: 0.8971 - val_accuracy: 0.5000 - val_loss

In [95]:
test_loss12, test_accuracy12 = model12.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy12)
print("Test Loss:", test_loss12)

Test Accuracy: 0.6333333253860474
Test Loss: 0.6761837005615234


In [102]:
model13 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=False),
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model13.summary()

Model: "sequential_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_68 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_69 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_80 (Dense)                     │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_75 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_81 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_76 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_82 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,676,801 (6.40 MB)

 Trainable params: 1,675,777 (6.39 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [103]:
model13.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model13.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 106ms/step - accuracy: 0.4929 - loss: 0.8330 - val_accuracy: 0.4667 - val_loss: 0.6936
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.5250 - loss: 0.8329 - val_accuracy: 0.4500 - val_loss: 0.6921
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.5679 - loss: 0.7844 - val_accuracy: 0.4500 - val_loss: 0.6908
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.5357 - loss: 0.8551 - val_accuracy: 0.4833 - val_loss: 0.6893
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.5750 - loss: 0.7975 - val_accuracy: 0.5000 - val_loss: 0.6878
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.4893 - loss: 0.8480 - val_accuracy: 0.4833 - val_loss: 0.6857
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.4893 - loss: 0.8804 - val_accuracy: 0.4833 - val_loss: 0.6840
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.6107 - loss: 0.7481 - val_accuracy: 0.5167 - val_loss

In [104]:
test_loss13, test_accuracy13 = model13.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy13)
print("Test Loss:", test_loss13)

Test Accuracy: 0.5333333611488342
Test Loss: 0.6699958443641663


* Adding more FC layers and more GRUs seems to improve the performance, but when the number of FCs exceed GRUs the performance drops.
* Including dropouts between GRU and FC layer and between the FCs, also result in better performing models, while including BatchNormalization gives mixed results.
* The best performing achitecture is one with 2 GRUs and 2 FCs with dropouts between GRU-FC and between FCs, with an accuracy of **66.7%** which is also the highest accuracy obtained among all the experimented models. The second highest accuracy seen is **63.3%** from the model having 3 GRUs with BatchNormalization between every pair and 2 FC layers with dropouts before and after each.

**It should also be noted that these models were extremely quick to train, which made trying out several different architectures very easy.**

* This architecture can be tuned further with the inclusion of different regularization parameters, different dropout rates, different optimizers, and momentum-based or scheduled learning rates.
  * Since, the training is fast as it is, adding momentum may not necessarily help.
  * Scheduling the learning rates and making it slower after a while may have higher scope of giving an improvement, even though all of the above trials used a small learning rate of 1e-5 (perhaps even smaller learning rates could help in this case).

* Furthermore, we could also try other pretrained models for obtaining the embeddings. But we need to take care of the sizes of the images that are fed into those models.
  * densenet, efficientnetb0, mobilenetv2, resnet50 -> 224x224
  * xception, inceptionv3 -> 299x299
  * efficientnetb3 -> 300x300

**Regardless of the model and technique used, we don't appear to get any high values of accuracy. This is due to the small size of the dataset and also because of the nature of the dataset. The deepfake videos aren't entirely AI-generated, instead the faces/expressions alone, of the people in the videos, have been swapped/altered. So, our model needs to identify the fakeness of the video from a very small spatial range of the frames. That is a sensitive task, and a model will only be able to handle that if it were fed a significantly large dataset to learn from.**